# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
df=pd.read_csv('cities.csv').drop('Unnamed: 0',axis =1)
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saint george,37.10,-113.58,46.994,15,1,3.60,US,1603817967
1,dikson,73.51,80.55,10.418,96,100,7.79,RU,1603818042
2,torbay,47.67,-52.73,41.000,100,90,4.10,CA,1603818179
3,castro,-24.79,-50.01,77.954,41,21,4.25,BR,1603818180
4,santa isabel do rio negro,-0.41,-65.02,93.812,48,93,0.65,BR,1603818182
...,...,...,...,...,...,...,...,...,...
570,longhua,23.61,114.19,70.502,70,29,1.19,CN,1603818451
571,batticaloa,7.71,81.69,80.492,80,34,2.36,LK,1603818451
572,villa carlos paz,-31.42,-64.50,84.992,25,0,9.80,AR,1603818451
573,san andres,12.58,-81.70,86.000,79,90,3.60,CO,1603818396


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
fig = gmaps.figure()
fig.add_layer(gmaps.WeightedHeatmap(locations=df[['Lat','Lng']],weights=df['Humidity']))
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#A max temperature lower than 80 degrees but higher than 70.

#Wind speed less than 4.75 m/s.

#Cloudiness less than 20

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
hotel_df=df[(df['Max Temp']<80)&(df['Max Temp']>70)&(df['Wind Speed']<4.75) &(df['Cloudiness']<20)].copy()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,bozova,37.36,38.53,71.600,19,0,2.10,TR,1603818382
73,santa fe,-31.63,-60.70,78.998,30,0,2.68,AR,1603818385
77,pyay,18.82,95.22,76.136,88,18,0.77,MM,1603818386
102,sibolga,1.74,98.78,76.892,78,10,1.45,ID,1603818329
104,grand gaube,-20.01,57.66,73.994,73,10,1.34,MU,1603818388
122,gat,31.61,34.76,78.008,100,0,2.85,IL,1603818184
123,tabas,33.60,56.92,71.600,19,0,2.10,IR,1603818127
170,avera,33.19,-82.53,75.200,78,1,1.75,US,1603818320
220,kihei,20.79,-156.47,78.800,74,1,0.98,US,1603818407
286,nueve de julio,-35.44,-60.88,73.994,44,4,2.68,AR,1603818417


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
target_search = "hotel"
target_radius = 5000
target_type = "hotel"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
names = []
# set up a parameters dictionary


for index,row in hotel_df.iterrows():
    target_coordinates = f'{row["Lat"]},{row["Lng"]}'
    params = {  "location": target_coordinates,
                "keyword": target_search,
                "radius": target_radius,
                "type": target_type,
                "key": g_key}
    r = requests.get(base_url, params=params)
    try:
        names.append(r.json['results'][0]['name'])
    except: #no hotels
        names.append(np.nan)

hotel_df['Hotel Name'] = names


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))